# 利用标注好的新浪微博数据 (weibo_senti_6k.csv，字段描述见表1)，开展情感分析
# 1. 数据集介绍
数据字段及具体含义如下：
- label 1 代表积极情感，0 代表消极情感
- review 微博内容

# 2. 实验准备

In [1]:
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np
import warnings
warnings.simplefilter("ignore")

## 3.1. 载入预处理后的数据

In [2]:
reviews = pd.read_csv("IMDB Dataset_with_tags.csv")
print ("# of reviews: ", reviews.shape[0])
reviews_label_count = reviews.groupby("sentiment").count()
print(reviews_label_count)
reviews.head()
reviews['sentiment'] = reviews['sentiment'].replace({'positive': 1, 'negative': 0})
print(reviews.head())

# of reviews:  50000
           review  noun_phrases
sentiment                      
negative    25000         25000
positive    25000         25000
                                              review  sentiment  \
0  One of the other reviewers has mentioned that ...          1   
1  A wonderful little production. <br /><br />The...          1   
2  I thought this was a wonderful way to spend ti...          1   
3  Basically there's a family where a little boy ...          0   
4  Petter Mattei's "Love in the Time of Money" is...          1   

                                        noun_phrases  
0  ['oz episode youll', 'mebr br', 'show pulls', ...  
1  ['wonderful little production br br', 'oldtime...  
2  ['wonderful way', 'hot summer weekend', 'seria...  
3  ['boy jake', 'timebr br', 'soap opera', 'zombi...  
4  ['petter matteis', 'mr mattei', 'vivid portrai...  


## 3.2. 划分训练集与测试集

In [3]:
X = []
for index, row in reviews.iterrows():
    review = row['review']
    X.append(review)
y = reviews["sentiment"]

from sklearn.model_selection import train_test_split
# Splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 618)
print(len(X_train), len(X_test))

40000 10000


# 4. 实验设计
## 4.1. 评论情感分析 - 向量空间模型
### 4.1.1. 词频向量 + 逻辑回归

In [4]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [5]:
tf_vec = CountVectorizer()
X_train_tf = tf_vec.fit_transform(X_train)
X_test_tf = tf_vec.transform(X_test)
print(X_train_tf.shape, X_test_tf.shape)

(40000, 93007) (10000, 93007)


In [6]:
# Fitting a logistic regression model with default parameters
logreg = LogisticRegression()
logreg.fit(X_train_tf, y_train)
# Prediction & Evaluation
y_hat_test = logreg.predict(X_test_tf)
print("Precision: {:.3f}".format(precision_score(y_test, y_hat_test)), "Recall: {:.3f}".format(recall_score(y_test, y_hat_test)))
print("F1 score: {:.3f}".format(f1_score(y_test, y_hat_test)))
y_hat_test = logreg.predict_proba(X_test_tf)
print("AUC score: {:.3f}".format(roc_auc_score(y_test, y_hat_test[:,1])))

Precision: 0.887 Recall: 0.896
F1 score: 0.892
AUC score: 0.951


### 4.1.2. TF-IDF向量 + 逻辑回归

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer()
X_train_tfidf = tfidf_vec.fit_transform(X_train)
X_test_tfidf = tfidf_vec.transform(X_test)
print(X_train_tfidf.shape, X_test_tfidf.shape)

(40000, 93007) (10000, 93007)


In [8]:
tfidf_vec.vocabulary_

{'seeing': 72739,
 'this': 82532,
 'movie': 54903,
 'always': 3630,
 'reminds': 68015,
 'me': 51983,
 'of': 58096,
 'what': 90284,
 'remember': 67994,
 'summer': 79699,
 'being': 8333,
 'like': 48129,
 'god': 34525,
 'such': 79480,
 'long': 48835,
 'time': 82977,
 'ago': 2775,
 'the': 82271,
 'entire': 27316,
 'scene': 71832,
 'involving': 42550,
 'overnight': 59457,
 'from': 32304,
 'canoe': 13065,
 'procession': 64492,
 'to': 83239,
 'end': 26956,
 'trip': 84586,
 'is': 42740,
 'precious': 63867,
 'and': 4084,
 'tripper': 84601,
 'story': 78674,
 'about': 1642,
 'homicidal': 38958,
 'maniac': 50575,
 'urban': 87399,
 'legend': 47499,
 'right': 69274,
 'out': 59094,
 'my': 55524,
 'youth': 92344,
 'highly': 38311,
 'entertaining': 27287,
 'made': 49900,
 'many': 50740,
 'times': 83003,
 'better': 8887,
 'by': 12474,
 'awful': 6526,
 'sequels': 73116,
 'that': 82251,
 'followed': 31308,
 'house': 39425,
 'dripped': 24927,
 'blood': 9810,
 'third': 82515,
 'in': 40884,
 'series': 73187,

In [9]:
# Fitting a logistic regression model with default parameters
logreg = LogisticRegression()
logreg.fit(X_train_tfidf, y_train)
# Prediction & Evaluation
y_hat_test = logreg.predict(X_test_tfidf)
print("Precision: {:.3f}".format(precision_score(y_test, y_hat_test)), "Recall: {:.3f}".format(recall_score(y_test, y_hat_test)))
print("F1 score: {:.3f}".format(f1_score(y_test, y_hat_test)))
y_hat_test = logreg.predict_proba(X_test_tfidf)
print("AUC score: {:.3f}".format(roc_auc_score(y_test, y_hat_test[:,1])))

Precision: 0.886 Recall: 0.908
F1 score: 0.897
AUC score: 0.959


### 4.1.3. 使用 TextBlob 自带的 sentiment 函数

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# 1. 加载数据
df = pd.read_csv('IMDB_Dataset_with_TextBlob_sentiment.csv')

# 2. 创建标签映射，确保 'sentiment' 的真实标签格式一致
label_mapping = {'positive': 1, 'negative': 0}
df['sentiment_mapped'] = df['sentiment'].replace(label_mapping)

# 3. 按 4:1 划分训练集和测试集（80% 训练集，20% 测试集）
_, X_test, y_train, y_test = train_test_split(
    df.index,  # 使用索引进行划分，因为我们只关心标签和情感分数
    df['sentiment_mapped'],  # 使用转换后的情感标签作为目标变量
    test_size=0.2,  # 20% 测试集，80% 训练集
    random_state=618  # 设置随机种子，确保每次划分相同
)

# 获取与测试集对应的TextBlob预测的情感分数
y_score_test = df.loc[X_test, 'textblob_sentiment_score']

# 根据情感分数生成二值预测结果，这里假设情感分数 > 0 表示正向评价
threshold = 0  # 设定阈值以区分正负评价
y_pred_test = (y_score_test > threshold).astype(int)

# 5. 评估：Precision, Recall, F1, AUC
precision = precision_score(y_test, y_pred_test)
recall = recall_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)
auc = roc_auc_score(y_test, y_score_test)  # 使用情感分数计算AUC

# 输出评估结果
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1 score: {f1:.3f}")
print(f"AUC score: {auc:.3f}")

Precision: 0.625
Recall: 0.945
F1 score: 0.752
AUC score: 0.833
